In [1]:
import cv2
import numpy as np
import os
from pyttsx3 import speak
from os import listdir
from os.path import isfile, join
import time

### model : face_classifier

In [2]:
if not(os.path.exists("./haarcascade_frontalface_default.xml")):
    os.system("curl https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml -O")
    print("Haarcascade file downloaded....")

In [3]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

### Cropped Face Image


In [4]:
def face_extractor(img):
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if len(faces)==0:return None
    
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

### collecting face samples

In [5]:
speak("enter your name")
name = input("Enter your name: ")

if not(os.path.exists("./faces")):
    os.mkdir("./faces")

directory = "./faces/{}".format(name) #dest dir
if not(os.path.exists(directory)):
    os.mkdir(directory)

    
cap = cv2.VideoCapture(0)
# url='http://192.168.76.148/capture'
count = 0

while True:

    ret, frame = cap.read()
#     image_resp=urllib.request.urlopen(url) 
#     arr = image_resp.read()
#     nparr = np.asarray(bytearray(arr),dtype=np.uint8)
#     frame = cv2.imdecode(nparr,cv2.IMREAD_COLOR)
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        file_name_path = directory + '/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        cv2.putText(face, str(count), (70, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)
        if count==1:print("start")
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found .Look into the camera")
        pass

    if cv2.waitKey(1) == 13 or count == 200: 
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

Enter your name: suhag
start


# Training the model

In [6]:
if not(os.path.exists("./faces/.ipynb_checkpoints")):os.mkdir("./faces/.ipynb_checkpoints")
os.listdir("faces/")[1:]

['suhag']

In [7]:
Training_Data, Labels = [], []
Dict = {}


for i,name in enumerate(os.listdir('faces/')[1:]):
    data_path = './faces/{}/'.format(name)
    onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
    Dict[i] = name
    

    for j, files in enumerate(onlyfiles):
        image_path = data_path + onlyfiles[j]
        images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        Training_Data.append(np.asarray(images, dtype=np.uint8))
        Labels.append(i)
#         Labels.append(name)

print(Dict)    
# print(Labels) 

Labels = np.asarray(Labels, dtype=np.int32)
#print(Labels)


Suhag_model  = cv2.face_LBPHFaceRecognizer.create()
Suhag_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully and saved..................")
if not(os.path.exists("./model")):os.mkdir("./model")
Suhag_model.write("./model/Suhag.yml")

{0: 'suhag'}
Model trained sucessefully and saved..................


# Recgonise and do the tasks

In [8]:
Dict={}
for i,name in enumerate(os.listdir('faces/')[1:]):
        Dict[i] = name
Dict

{0: 'suhag'}

In [9]:
# time_hour = time.localtime().tm_hour
# time_min = time.localtime().tm_min



face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\suhag\AppData\Local\Temp\ipykernel_6188\3728086114.py:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


## Emotion Detection

In [10]:
from deepface import DeepFace

In [11]:
def detect_emotion(image):
    res = DeepFace.analyze(image,actions=('emotion'),silent=True,align=False)
    emotion = res[0]['dominant_emotion']
    region  = res[0]['region']
    x = region['x']
    y = region['y']
    w = region['w']
    h = region['h']
#     cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,255),2)
    cv2.putText(image,emotion,(x+w,y+h),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,0),2)

In [12]:
def face_recognise(frame):
    image, face = face_detector(frame)
    try:
#         detect_emotion(image)
        res = DeepFace.analyze(image,actions=('emotion'),silent=True,align=False)
        emotion = res[0]['dominant_emotion']
        region  = res[0]['region']
        x = region['x']
        y = region['y']
        w = region['w']
        h = region['h']
    #     cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,255),2)
        cv2.putText(image,emotion,(x+w,y+h),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,0),2)
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        results = Priyanka_model.predict(face)
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident you are Priyanka!!'  
        #cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        person = results[0]
        
        if confidence > 70:
            cv2.putText(image, "Hey {}!!!!".format(Dict[person]), (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
#             pyttsx3.speak("This is {}".format(Dict[person]))
            return image,Dict[person],emotion
        else:
            cv2.putText(image, "I can not recognise you.", (200, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            return image,None,None
    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        return image,None,None

# YOLO code

In [13]:
import torch
import cv2

In [14]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in C:\Users\suhag/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-3-28 Python-3.9.13 torch-2.0.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


## Object detection model

In [15]:
def detect_objects(image):
#     obj_names = {}
    obj_names = []
    results = model(image)
    objects = results.crop(save=False)
    for obj in objects:
#             obj_names[obj["label"].split()[0]] = obj["label"].split()[1]
        if float(obj["label"].split()[-1])>0.65:obj_names.append(" ".join(obj["label"].split()[:-1]))
    obj_box_pics = results.render()[0]
    obj_names_count={}
    for i in set(obj_names):
        obj_names_count[i] = obj_names.count(i)
    return (obj_names_count,obj_box_pics)

In [16]:
Priyanka_model = cv2.face.LBPHFaceRecognizer_create()
Priyanka_model.read('./model/Priyanka.yml')

In [18]:
cam = cv2.VideoCapture(0)
# url='http://192.168.76.148/capture'
prev_obj=[None]
prev_person_name = None
prev_emotion = None
while True:
    clicked , image = cam.read()
#     clicked = True
#     image_resp=urllib.request.urlopen(url) 
#     arr = image_resp.read()
#     nparr = np.asarray(bytearray(arr),dtype=np.uint8)
#     image = cv2.imdecode(nparr,cv2.IMREAD_COLOR)
    
    if clicked:
        obj_names,obj_box_pics = detect_objects(image)
#         print(obj_names)
        if not(prev_obj==obj_names): 
            prev_obj = obj_names
            for obj,count in obj_names.items():
                speak("{} {}".format(count,obj))
        if "person" in obj_names:
            obj_box_pics,person_name,emotion = face_recognise(obj_box_pics)
            if not(prev_person_name==person_name) or not(prev_emotion==emotion):
                prev_person_name = person_name
                prev_emotion = emotion
                if person_name:
                    speak("This is {} and emotion is {}".format(person_name,emotion))
        cv2.imshow("object_detected",obj_box_pics)
        

    if cv2.waitKey(1)==13:
        break
cv2.destroyAllWindows()

cam.release()